In [4]:
# If you have installation questions, please reach out

import pandas as pd # data storage
import xgboost as xgb # graident boosting 
import numpy as np  # math and stuff
import matplotlib.pyplot as plt # plotting utility
import sklearn # ML and stats

import datetime

print('XGBoost ver:', xgb.__version__)
print('scikit ver:', sklearn.__version__)

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import  max_error, mean_squared_error, median_absolute_error
from sklearn.model_selection import GridSearchCV

XGBoost ver: 1.4.2
scikit ver: 0.24.2


In [5]:
import defaults
from defaults import framecleaner, splitterz

# dataframes

In [6]:
df0  = pd.read_csv('../../core_to_wl_merge/OS0_Merged_dataset_imputed_08_23_2021.csv')

df1 = pd.read_csv('../../core_to_wl_merge/OS1_Merged_dataset_imputed_08_23_2021.csv')

df2 = pd.read_csv('../../core_to_wl_merge/OS2_Merged_dataset_imputed_08_23_2021.csv')

C:\Users\tmartin\.conda\envs\lewisml\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Parameters

In [7]:
param_dict ={
    "tree_type": "auto", #Auto or exact, or hist will all work
    "dataset": ['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE', 'RESD', 'PHIN', 'PHID', 'GR_smooth',  'PE_smooth', 'He_por'],
    "inputs": ['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE', 'RESD', 'PHIN', 'PHID', 'GR_smooth',  'PE_smooth'],
    "target": ['He_por']}

In [8]:
def xgb_por(X_train, X_test, y_train, y_test, OS='os', tree_type='auto'):
  xg_reg = xgb.XGBRegressor()

  xg_reg.fit(X_train,y_train)

  preds = xg_reg.predict(X_test)

  rmse = mean_squared_error(y_test, preds, squared=False)
  print("Root Mean Squared Error: %f" % (rmse))
  max = max_error(y_test, preds)
  print("Max Error: %f" % (max))
  MAE = median_absolute_error(y_test, preds)
  print("Median Abs Error: %f" % (MAE))

  parameters = {
    'max_depth': range (4, 6, 1),
    'n_estimators': range(20, 55, 5),
    'colsample_bytree': [ 0.9, 1],
    'learning_rate': [ 0.2, 0.1, 0.05],
    'max_delta_step':  [ 0, 1, 2, 4, 8, 16],
    'reg_alpha' : [0, 1, 2, 4]
}

  estimator = xgb.XGBRegressor(tree_method=tree_type)
                             # gpu_id=0) # uses the GPU
  # Grid Search
  grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    n_jobs = 8,
    cv = 5,
    verbose = 1)

  grid_search.fit(X_train, y_train)

  model1 = xgb.XGBRegressor(
                          tree_method=tree_type,
                          n_estimators=grid_search.best_estimator_.n_estimators, 
                          max_depth = grid_search.best_estimator_.max_depth,
                          learning_rate=grid_search.best_estimator_.learning_rate, 
                          colsample_bytree=grid_search.best_estimator_.colsample_bytree)

  model1.fit(X_train, y_train)

  preds2 = model1.predict(X_test)

  rmse2 = mean_squared_error(y_test, preds2, squared=False)
  print("Mean Squared Error: %f" % (rmse2))
  max1 = max_error(y_test, preds2)
  print("Max Error: %f" % (max1))
  MAE2 = median_absolute_error(y_test, preds2)
  print("Median Abs Error: %f" % (MAE2))

  x = datetime.datetime.now()

  d = {'target': ['por', 'por'],
     'MSE': [rmse, rmse2],
     'MAE': [MAE, MAE2],
     'MaxError': [max, max1], 
     'day': [x.day, x.day], 
     'month':[x.month, x.month], 
     'year':[x.year, x.year],
     'model':['XGB', 'XGB'],
     'version':[xgb.__version__, xgb.__version__ ]}

  results = pd.DataFrame(data=d)

  return results

# Offset 0

In [10]:
df0 = df0[df0.He_por >= 0]
df0 = df0[df0.USGS_ID != 'E997'] # removing E997

#Create the dataset
X, Y_array = framecleaner(df0, param_dict['dataset'], param_dict['inputs'], param_dict['target'] )

#Split the dataset
X_train, X_test, y_train, y_test = splitterz(X.values, Y_array)

df_OS0 = xgb_por(X_train, X_test, y_train, y_test, OS='OS0')

C:\Users\tmartin\.conda\envs\lewisml\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Root Mean Squared Error: 1.558809
Max Error: 6.401019
Median Abs Error: 0.683291
Fitting 5 folds for each of 2016 candidates, totalling 10080 fits
Mean Squared Error: 1.582253
Max Error: 7.386353
Median Abs Error: 0.717187


In [11]:
df_OS0

,target,MSE,MAE,MaxError,day,month,year,model,version
0,por,1.558809,0.683291,6.401019,5,9,2021,XGB,1.4.2
1,por,1.582253,0.717187,7.386353,5,9,2021,XGB,1.4.2


# Offset 1

In [11]:
df1 = df1[df1.He_por >= 0]
df1 = df1[df1.USGS_ID != 'E997'] # removing E997

#Create the dataset
X, Y1_array = framecleaner(df1, param_dict['dataset'], param_dict['inputs'], param_dict['target'] )

#Split the dataset
X_train, X_test, y_train, y_test = splitterz(X.values, Y_array)

df_OS0 = xgb_por(X_train, X_test, y_train, y_test, OS='OS0')

In [12]:
xgb_por(X_train2, X_test2, y_train2, y_test2, 'OS1_xgb_por.csv')

Root Mean Squared Error: 1.598138
Max Error: 7.133600
Median Abs Error: 0.678442
Fitting 5 folds for each of 2016 candidates, totalling 10080 fits
Mean Squared Error: 1.578548
Max Error: 5.788961
Median Abs Error: 0.669650


,target,MSE,MAE,MaxError,day,month,year,model,version
0,[He_por],1.598138,0.678442,7.133600,25,8,2021,XGB,1.4.2
1,[He_por],1.578548,0.669650,5.788961,25,8,2021,XGB,1.4.2


# Offset 2

In [13]:
df3 = df3.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

df3 = df3[df3.He_por >= 0]
df3 = df3[df3.USGS_ID != 'E997'] # removing E997

dataset3 = df3[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth','He_por']]

# Features we will use for prediction
X3 = dataset3[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y3 = dataset3[['He_por']]

Y_array3 = np.array(Y3.values)

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3.values, Y_array3, test_size=test_size)

In [14]:
xgb_por(X_train3, X_test3, y_train3, y_test3, 'OS2_xgb_por.csv')

Root Mean Squared Error: 1.829887
Max Error: 7.319721
Median Abs Error: 0.647403
Fitting 5 folds for each of 2016 candidates, totalling 10080 fits
Mean Squared Error: 1.692846
Max Error: 6.297692
Median Abs Error: 0.563098


,target,MSE,MAE,MaxError,day,month,year,model,version
0,[He_por],1.829887,0.647403,7.319721,25,8,2021,XGB,1.4.2
1,[He_por],1.692846,0.563098,6.297692,25,8,2021,XGB,1.4.2
